## Dependencies

In [8]:
import wikipedia
from tqdm import tqdm

## Data collection

Sentences for both dialects will be collected from summaries of random wikipedia pages.

In [7]:
NUM_PAGES_WESTERN = 50
NUM_PAGES_EASTERN = 50

In [9]:
western_text = []
eastern_text = []

# Western Armenian
wikipedia.set_lang('hyw')
for i in tqdm(range(NUM_PAGES_WESTERN)):
    title = wikipedia.random()
    page = wikipedia.search(title)
    western_text.append(wikipedia.summary(page))
    
# Eastern Armenian
wikipedia.set_lang('hy')
for i in tqdm(range(NUM_PAGES_EASTERN)):
    title = wikipedia.random()
    page = wikipedia.search(title)
    eastern_text.append(wikipedia.summary(page))

100%|██████████| 50/50 [01:31<00:00,  1.83s/it]


TODO:

-clean up texts (remove punctuation)

-remove non-Armenian words

Create corresponding labels and join texts.